# Loan Denial data cleaning

In [614]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [615]:
lc_accepted=pd.read_csv('LC_Data/Lending_Club_Accepted_2014_2018.csv')
lc_rejected=pd.read_csv('LC_Data/Lending_Club_Rejected_2014_2018.csv')
lc_common=[ 'loan_amnt', 'issue_d', 'purpose', 'dti', 'addr_state', 'emp_length', 'policy_code']
lc_accepted=lc_accepted[lc_common]
lc_accepted.head()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_1110/1814094668.py:1: DtypeWarning: Columns (19,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  lc_accepted=pd.read_csv('LC_Data/Lending_Club_Accepted_2014_2018.csv')


,loan_amnt,issue_d,purpose,dti,addr_state,emp_length,policy_code
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0


In [616]:
lc_rejected.drop(['Zip_Code', 'Risk_Score'], axis=1, inplace=True)
lc_rejected.head()
print(lc_rejected['Debt_to_Income_Ratio'])

0            2.69%
1           28.26%
2              -1%
3           21.43%
4            8.49%
             ...  
26132303    41.26%
26132304     1.48%
26132305    10.26%
26132306    17.71%
26132307    10.58%
Name: Debt_to_Income_Ratio, Length: 26132308, dtype: object


In [617]:
rename_dict={'addr_state':'State', 'loan_amnt':'Amount_Requested', 'issue_d':'Application_Date', 'purpose':'Loan_Title', 'dti':'Debt_to_Income_Ratio', 'emp_length':'Employment_Length', 'policy_code':'Policy_Code'}
lc_accepted.rename(columns=rename_dict, inplace=True)
lc_accepted.head()

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0


In [618]:
#create column to indicate loans were accepted or rejected
lc_accepted['Accepted']=1
lc_accepted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029952 entries, 0 to 2029951
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 123.9+ MB


In [619]:
lc_rejected['Accepted']=0
lc_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26132308 entries, 0 to 26132307
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  object 
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(2), int64(1), object(5)
memory usage: 1.6+ GB


In [620]:
#convert Debt_to_Income_Ratio to numeric in lc_rejected
lc_rejected['Debt_to_Income_Ratio']=pd.to_numeric(lc_rejected['Debt_to_Income_Ratio'].str.strip('%'))
lc_rejected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26132308 entries, 0 to 26132307
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.6+ GB


In [621]:
#rename values in Loan_Title column to be same for lc_accepted and lc_rejected
loan_map={'Debt consolidation': 'debt_consolidation', 'Home improvement':'home_improvement', 'Home buying':'house', 'Other':'other', 'Credit card refinancing':'credit_card', 'Major purchase':'major_purchase', 'Business Loan':'small_business', 'Business':'small_business', 'Vacation':'vacation', 'Car financing':'car', 'Moving and relocation':'moving', 'Medical expenses':'medical', 'Green loan':'small_business'}
lc_rejected['Loan_Title']=lc_rejected['Loan_Title'].replace(loan_map)

In [622]:
#append lc_accepted and lc_rejected to create a single dataframe
lc_all=lc_accepted.append(lc_rejected, ignore_index=True)
lc_all['Employment_Length'].fillna(0, inplace=True)
lc_all.dropna(inplace=True)
lc_all.info()

/var/folders/d3/ry6v0r7j28d4wdqwrz_hvdqc0000gn/T/ipykernel_1110/2391448617.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  lc_all=lc_accepted.append(lc_rejected, ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 28158341 entries, 0 to 28162259
Data columns (total 8 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Amount_Requested      float64
 1   Application_Date      object 
 2   Loan_Title            object 
 3   Debt_to_Income_Ratio  float64
 4   State                 object 
 5   Employment_Length     object 
 6   Policy_Code           float64
 7   Accepted              int64  
dtypes: float64(3), int64(1), object(4)
memory usage: 1.9+ GB


In [623]:
lc_all.head()

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted
0,3600.0,Dec-2015,debt_consolidation,5.91,PA,10+ years,1.0,1
1,24700.0,Dec-2015,small_business,16.06,SD,10+ years,1.0,1
2,20000.0,Dec-2015,home_improvement,10.78,IL,10+ years,1.0,1
3,35000.0,Dec-2015,debt_consolidation,17.06,NJ,10+ years,1.0,1
4,10400.0,Dec-2015,major_purchase,25.37,PA,3 years,1.0,1


In [624]:
lc_all= lc_all[lc_all.groupby('Loan_Title').Loan_Title.transform('count')>10000].copy()
print(lc_all['Loan_Title'].value_counts())

debt_consolidation    12889660
other                  4603983
credit_card            3979156
home_improvement       1247525
car                    1243447
major_purchase          937487
small_business          836718
medical                 736544
moving                  677696
house                   668817
vacation                305394
renewable_energy         27969
Name: Loan_Title, dtype: int64


In [625]:
#sort lc_all by state
lc_all.sort_values(by='State', inplace=True)
lc_all.head(1000)

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted
8225641,8000.0,2018-09-29,car,2.50,AK,< 1 year,0.0,0
19034336,20500.0,2018-05-17,credit_card,72.48,AK,< 1 year,0.0,0
19034348,2000.0,2018-05-17,other,1.58,AK,< 1 year,0.0,0
21303110,10000.0,2017-11-16,moving,4.58,AK,< 1 year,0.0,0
10103547,1000.0,2014-12-15,moving,23.56,AK,< 1 year,0.0,0
...,...,...,...,...,...,...,...,...
27739231,5000.0,2016-12-04,house,6.79,AK,5 years,0.0,0
14214739,5000.0,2016-01-13,car,24.40,AK,< 1 year,0.0,0
28081210,5000.0,2016-12-26,other,35.95,AK,5 years,0.0,0
11982661,100000.0,2018-03-27,small_business,-1.00,AK,< 1 year,0.0,0


## Add the categorization of the states to the dataframe

In [626]:
#import the state categories from the csv file
devscaled=pd.read_csv('devscaled.csv')
devscaled.set_index('State', inplace=True)
devscaled.head()

,2014_gdp,2015_gdp,2016_gdp,2017_gdp,2018_gdp,2014_unemp,2015_unemp,2016_unemp,2017_unemp,2018_unemp,...,2014_rank,2015_rank,2016_rank,2017_rank,2018_rank,2014,2015,2016,2017,2018
State,,,,,,,,,,,,,,,,,,,,,
AL,0.072278,0.069670,0.068742,0.066898,0.065489,-0.788462,-0.804878,-0.725,-0.434783,-0.357143,...,41.0,43.0,44.0,42.0,44.0,3,3,3,3,3
AK,0.011349,0.011004,0.010356,0.009803,0.009041,-0.807692,-0.902439,-1.000,-1.000000,-1.000000,...,51.0,51.0,51.0,51.0,51.0,3,3,3,3,3
AZ,0.111370,0.108858,0.109180,0.108649,0.108648,-0.788462,-0.804878,-0.625,-0.543478,-0.571429,...,48.0,46.0,46.0,48.0,48.0,3,3,3,3,3
AR,0.037188,0.035747,0.034768,0.033589,0.033023,-0.634615,-0.536585,-0.275,-0.282609,-0.309524,...,39.0,36.0,28.0,31.0,36.0,3,3,2,2,3
CA,1.000000,1.000000,1.000000,1.000000,1.000000,-0.923077,-0.829268,-0.650,-0.521739,-0.428571,...,5.0,2.0,2.0,1.0,1.0,1,1,1,1,1


In [627]:
devscaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, AL to WY
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   2014_gdp     51 non-null     float64
 1   2015_gdp     51 non-null     float64
 2   2016_gdp     51 non-null     float64
 3   2017_gdp     51 non-null     float64
 4   2018_gdp     51 non-null     float64
 5   2014_unemp   51 non-null     float64
 6   2015_unemp   51 non-null     float64
 7   2016_unemp   51 non-null     float64
 8   2017_unemp   51 non-null     float64
 9   2018_unemp   51 non-null     float64
 10  2018_health  51 non-null     float64
 11  2017_health  51 non-null     float64
 12  2016_health  51 non-null     float64
 13  2015_health  51 non-null     float64
 14  2014_health  51 non-null     float64
 15  2018_edu     51 non-null     float64
 16  2017_edu     51 non-null     float64
 17  2016_edu     51 non-null     float64
 18  2015_edu     51 non-null     float64
 19  2014_edu     5

In [628]:
lc_all['Year']=lc_all['Application_Date'].str.extract('(\d{4})', expand=False)
lc_all.head()

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted,Year
8225641,8000.0,2018-09-29,car,2.50,AK,< 1 year,0.0,0,2018
19034336,20500.0,2018-05-17,credit_card,72.48,AK,< 1 year,0.0,0,2018
19034348,2000.0,2018-05-17,other,1.58,AK,< 1 year,0.0,0,2018
21303110,10000.0,2017-11-16,moving,4.58,AK,< 1 year,0.0,0,2017
10103547,1000.0,2014-12-15,moving,23.56,AK,< 1 year,0.0,0,2014


In [629]:
#add a State_Category column to the lc_all dataframe by the year and the state in the devscaled dataframe
lc_all['State_Category']=lc_all.apply(lambda x: devscaled.loc[x['State'], x['Year']], axis=1)


In [630]:
lc_all.head(10000)

,Amount_Requested,Application_Date,Loan_Title,Debt_to_Income_Ratio,State,Employment_Length,Policy_Code,Accepted,Year,State_Category
8225641,8000.0,2018-09-29,car,2.50,AK,< 1 year,0.0,0,2018,3
19034336,20500.0,2018-05-17,credit_card,72.48,AK,< 1 year,0.0,0,2018,3
19034348,2000.0,2018-05-17,other,1.58,AK,< 1 year,0.0,0,2018,3
21303110,10000.0,2017-11-16,moving,4.58,AK,< 1 year,0.0,0,2017,3
10103547,1000.0,2014-12-15,moving,23.56,AK,< 1 year,0.0,0,2014,3
...,...,...,...,...,...,...,...,...,...,...
22803384,15000.0,2015-04-01,debt_consolidation,3.82,AK,0,0.0,0,2015,3
10965813,3500.0,2018-02-06,debt_consolidation,23.41,AK,< 1 year,0.0,0,2018,3
18066642,10000.0,2018-04-13,debt_consolidation,49.44,AK,< 1 year,0.0,0,2018,3
14171438,2500.0,2016-01-10,debt_consolidation,14.52,AK,< 1 year,0.0,0,2016,3


In [631]:
#save rankings to a file
lc_all.to_csv('loan_denials_all.csv', index=False)